<a href="https://colab.research.google.com/github/hariskhan-hk/Churn_pred/blob/main/Decision_Tree_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [138]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [139]:
tel_df = pd.read_csv('/content/Cleaned_Telecom_Dataset.csv', index_col = 0)
tel_df.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,True,False,False,True,True,False,...,False,False,True,False,True,False,False,False,False,False
1,0,56.95,1889.50,0,False,True,True,False,True,False,...,False,False,False,True,False,False,True,False,False,False
2,0,53.85,108.15,1,False,True,True,False,True,False,...,False,False,False,True,True,False,False,False,False,False
3,0,42.30,1840.75,0,False,True,True,False,True,False,...,True,False,False,False,False,False,False,True,False,False
4,0,70.70,151.65,1,True,False,True,False,True,False,...,False,False,True,False,True,False,False,False,False,False


In [140]:
x = tel_df.drop('Churn', axis = 1)
y = tel_df['Churn']

Train Test Split

In [141]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

With Python Package

In [142]:
model_dt= DecisionTreeClassifier(criterion = "entropy",random_state = 42,max_depth=6, min_samples_leaf=8)

In [143]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=8,
                       random_state=42)

In [144]:
y_pred= model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [145]:
model_dt.score(x_test,y_test)

0.7746979388770433

In [146]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.81      0.89      0.85      1001
           1       0.64      0.49      0.56       406

    accuracy                           0.77      1407
   macro avg       0.73      0.69      0.70      1407
weighted avg       0.76      0.77      0.76      1407



As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [147]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [148]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [149]:
model_dt_smote=DecisionTreeClassifier(criterion = "entropy",random_state = 42,max_depth=6, min_samples_leaf=8)

In [150]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9127118644067796
              precision    recall  f1-score   support

           0       0.95      0.86      0.90       546
           1       0.89      0.96      0.92       634

    accuracy                           0.91      1180
   macro avg       0.92      0.91      0.91      1180
weighted avg       0.92      0.91      0.91      1180



In [151]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[470  76]
 [ 27 607]]


Now we can see quite better results, i.e. Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.